<a href="https://colab.research.google.com/github/jsacco1/ml/blob/master/skin_lesion_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal:
Submit automated predictions of disease classification within dermoscopic images.

Possible disease categories are:


1. Melanoma
2. Melanocytic nevus
3. Basal cell carcinoma
4. Actinic keratosis / Bowen’s disease (intraepithelial carcinoma)
5. Benign keratosis (solar lentigo / seborrheic keratosis / lichen planus-like keratosis)
6. Dermatofibroma
7. Vascular lesion

Data
Input Data
The input data are dermoscopic lesion images in JPEG format.

All lesion images are named using the scheme ISIC_.jpg, where  is a 7-digit unique identifier. EXIF tags in the images have been removed; any remaining EXIF tags should not be relied upon to provide accurate metadata.

The lesion images come from the HAM10000 Dataset, and were acquired with a variety of dermatoscope types, from all anatomic sites (excluding mucosa and nails), from a historical sample of patients presented for skin cancer screening, from several different institutions. Images were collected with approval of the Ethics Review Committee of University of Queensland (Protocol-No. 2017001223) and Medical University of Vienna (Protocol-No. 1804/2017).

The distribution of disease states represent a modified “real world” setting whereby there are more benign lesions than malignant lesions, but an over-representation of malignancies.

Response Data
The response data are sets of binary classifications for each of the 7 disease states, indicating the diagnosis of each input lesion image.

Response data are all encoded within a single CSV file (comma-separated value) file, with each classification response in a row.

image: an input image identifier of the form ISIC_


1.   image: an input image identifier of the form ISIC_
MEL: “Melanoma” diagnosis confidence

2.  MEL: “Melanoma” diagnosis confidence
3.  NV: “Melanocytic nevus” diagnosis confidence
4. BCC: “Basal cell carcinoma” diagnosis confidence
5. AKIEC: “Actinic keratosis / Bowen’s disease (intraepithelial carcinoma)” diagnosis confidence
6. BKL: “Benign keratosis (solar lentigo / seborrheic keratosis / lichen planus-like keratosis)” diagnosis confidence
7. DF: “Dermatofibroma” diagnosis confidence
8. VASC: “Vascular lesion” diagnosis confidence

Diagnosis confidences are expressed as floating-point values in the closed interval [0.0, 1.0], where 0.5 is used as the binary classification threshold. Note that arbitrary score ranges and thresholds can be converted to the range of 0.0 to 1.0, with a threshold of 0.5, trivially using the following sigmoid conversion:

1 / (1 + e^(-(a(x - b))))
where x is the original score, b is the binary threshold, and a is a scaling parameter (i.e. the inverse measured standard deviation on a held-out dataset). Predicted responses should set the binary threshold b to a value where the classification system is expected to achieve 89% sensitivity, although this is not required.

Predicted diagnosis confidence values may vary independently, though exactly one disease state is actually present in each input lesion image.

Ground Truth Provenance

> As detailed in the HAM10000 Dataset description, diagnosis ground truth (provided for training and used internally for scoring validation and test phases) were established by one of the following methods:



1.   Histopathology
2.   Reflectance confocal microscopy
3.   Lesion did not change during digital dermatoscopic follow up over two years with at least three images
4.   Consensus of at least three expert dermatologists from a single image

In all cases of malignancy, disease diagnoses were histopathologically confirmed.

# Evaluation

Goal Metric
Predicted responses are scored using a normalized multi-class accuracy metric (balanced across categories). Tied positions will be broken using the area under the receiver operating characteristic curve (AUC) metric.

Rationale
Clinical application on skin lesion classification has two goals eventually: Giving specific information and treatment options for a lesion, and detecting skin cancer with a reasonable sensitivity and specificity. The first task needs a correct specific diagnosis out of multiple classes, whereas the second demands a binary decision “biopsy” versus “don’t biopsy”. In the former ISIC challenges, focus was on the second task, therefore this year we want to rank for the more ambitious metric of normalized multiclass accuracy, as it is also closer to real evaluation of a dermatologist. This is also important for the extending reader study, where the winning algorithm(s) will be compared to physicians performance in classification of digital images.

Other Metrics
Participants will be ranked and awards granted based only on the multiclass accuracy metric. However, for scientific completeness, predicted responses will also have the following metrics computed (comparing prediction vs. ground truth) for each image:

## Individual Category Metrics
1. sensitivity
2. specificity
3. accuracy
4. area under the receiver operating characteristic curve (AUC)
5. mean average precision
6. F1 score
7. AUC integrated between 80% to 100% sensitivity (AUC80) for Melanoma diagnosis only
8. positive predictive value (PPV)
9. negative predictive value (NPV)

## Aggregate Metrics
1. average AUC across all diagnoses
2. malignant vs. benign diagnoses category AUC

# Instructions

To participate in this task:

## 1. Train
Download the training input data and training ground truth response data.
Develop an algorithm for generating lesion diagnosis classifications in general.
## 2. Validate (optional)
Download the validation input data.
Run your algorithm on the validation Input data to produce validation predicted responses.
Submit these validation predicted responses to receive an immediate score. This will provide feedback that your predicted responses have the correct data format and have reasonable performance. You may make unlimited submissions.
## 3. Test
Download the test input data.
Run your algorithm on the test input data to produce test predicted responses.
Submit these test predicted responses. You may submit a maximum of 3 separate approaches/algorithms to be evaluated independently. You may make unlimited submissions, but only the most recent submission for each approach will be used for official judging. Use the “brief description of your algorithm’s approach” field on the submission form to distinguish different approaches. Previously submitted approaches are available in the dropdown menu.
Submit a manuscript describing your algorithm’s approach.

In [1]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import boto3

In [0]:
s3 = boto3.client('s3')

# then install data from s3 (create IAM role)

In [6]:
 !pip install awscli


In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
import os
os.chdir("/content/gdrive/My Drive/deep_learning_challenge")
os.getcwd()

'/content/gdrive/My Drive/deep_learning_challenge'

In [9]:
!pip install scipy==1.1.0

In [10]:
# Import libraries
from __future__ import print_function, division

import numpy as np
import random
import os
import glob
import cv2
import datetime
import pandas as pd
import time
import h5py
import csv

from scipy.misc.pilutil import imread, imsave

from sklearn.metrics import log_loss, confusion_matrix
from sklearn.utils import shuffle
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold
from PIL import Image, ImageChops, ImageOps

import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import RMSprop
from keras import backend as K
from keras.callbacks import EarlyStopping, Callback
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras import optimizers
from keras.models import Sequential, model_from_json
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, Activation, Dropout, Flatten, Dense, BatchNormalization
#Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
# use ImageDataGenerator to preprocess the data
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


Possible disease categories are:


1. Melanoma
2. Melanocytic nevus
3. Basal cell carcinoma
4. Actinic keratosis / Bowen’s disease (intraepithelial carcinoma)
5. Benign keratosis (solar lentigo / seborrheic keratosis / lichen planus-like keratosis)
6. Dermatofibroma
7. Vascular lesion

In [0]:
path = "/content/gdrive/My Drive/deep_learning_challenge/ISIC2018_Task3_Training_Input"
files = os.listdir(path)
